# LangChain Basics

## LangChain Main Concepts:

- LangChain Components
- Chains (Simple, Sequential, Custom)
- Agents

### Preliminary Part

In [ ]:
# installing the required libraries
!pip install -r ./requirements.txt -q

Le librerie da importare sono:
- openai
- langchain
- pinecone-client
- python-dotenv

In [ ]:
!pip show langchain

In [ ]:
# upgrading langchain
!pip install langchain --upgrade -q

### Importiamo le API Keys dal file _.env_ mediante: **python-dotenv**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

# loading the API Keys (OpenAI, Pinecone) from .env
load_dotenv(find_dotenv(), override=True)

os.environ.get('PINECONE_API_KEY')

## LangChain Components:
- LLM Wrappers (LLM and Chat)
- Prompt Templates
- Indexes
- Memory

## LLM Models (Wrappers): GPT-3

In [2]:
# Importo OpenAI e creo un'istanza di una LLM (in questo caso GPT-3)
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=512)
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'max_tokens': 512, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}


In [3]:
# Passo un prompt al LLM e ottengo una risposta
output = llm('explain quantum mechanics in one sentence')
print(output)



Quantum mechanics is the branch of physics that describes the behavior of matter and energy at the scale of atoms and subatomic particles.


In [4]:
# Verifico il mio prompt quanti token usa
print(llm.get_num_tokens('explain quantum mechanics in one sentence'))

7


In [5]:
# Richiesta multipla, mediante lista di prompt
output = llm.generate(['... is the capital of France.', 
                   'What is the formula for the area of a circle?'])

In [6]:
print(output.generations)

[[Generation(text='\n\nParis', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nThe formula for the area of a circle is A = πr², where A is the area, π is 3.14, and r is the radius of the circle.', generation_info={'finish_reason': 'stop', 'logprobs': None})]]


In [7]:
# Primo risultato
print(output.generations[0][0].text)



Paris


In [ ]:
# Secondo risultato
print(output.generations[1][0].text)

In [8]:
len(output.generations)

2

In [9]:
# Richiedo 3 possibili risposte per lo stesso prompt
output = llm.generate(['Write an orignal tagline for a burger restaurant'] * 3)

In [10]:
for o in output.generations:
    print(o[0].text, end='')



"Taste the Juiciness of our Burgers - Bite Into Burgers!"

"Making Mouth-Watering Burgers Since Day One!"

"Experience the Best Burgers in Town - At Burger Haven!"

## Chat Models (Wrappers): GPT-3.5-Turbo and GPT-4

In [11]:
# Importo lo "schema" dei messaggi
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [12]:
chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5, max_tokens=1024) #potrei usare anche 'gpt-4'
messages = [
    SystemMessage(content='You are a physicist and respond only in German.'),
    HumanMessage(content='explain quantum mechanics in one sentence')
]
output = chat(messages)

In [13]:
print(output.content)

Quantenmechanik beschreibt das Verhalten von Teilchen auf mikroskopischer Ebene und erlaubt es uns, ihre Eigenschaften und Wechselwirkungen zu verstehen.


In [14]:
chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5, max_tokens=1024) #potrei usare anche 'gpt-4'
messages = [
    SystemMessage(content='You are a physicist and respond only in Italian.'),
    HumanMessage(content='explain quantum mechanics in one sentence')
]
output = chat(messages)

In [15]:
print(output.content)

La meccanica quantistica è una teoria fisica che descrive il comportamento delle particelle subatomiche come onde probabilistiche, e che richiede l'uso di equazioni ondulatorie per calcolare le probabilità di misura di osservabili.


## Prompt Templates

In [17]:
from langchain import PromptTemplate

In [18]:
template = '''You are an experienced virologist.
Write a few sentences about the following {virus} in {language}.'''

prompt = PromptTemplate(
    input_variables=['virus', 'language'],
    template=template
)
print(prompt)

input_variables=['virus', 'language'] output_parser=None partial_variables={} template='You are an experienced virologist.\nWrite a few sentences about the following {virus} in {language}.' template_format='f-string' validate_template=True


In [19]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7)
output = llm(prompt.format(virus='ebola', language='Italian'))
print(output)



L'ebola è una malattia infettiva virale che causa una grave sindrome da sindrome emorragica. Si ritiene che sia stata originariamente trasmessa dai pipistrelli ai primati e poi all'uomo. I sintomi dell'ebola includono febbre alta, mal di testa, dolori muscolari, debolezza e diarrea. La malattia può diventare molto grave, portando a insufficienza renale, insufficienza epatica e sanguinamento interno e esterno.


## Simple Chains (single LLM used to perform a single task)

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)
template = '''You are an experienced virologist.
Write a few sentences about the following {virus} in {language}.'''

prompt = PromptTemplate(
    input_variables=['virus', 'language'],
    template=template
)

chain = LLMChain(llm=llm, prompt=prompt)
output = chain.run({'virus': 'HSV', 'language': 'italian'})
# Nel caso di un solo argomento non serve il dizionario
#output = chain.run('HSV')


In [21]:
print(output)

L'HSV, acronimo di Herpes Simplex Virus, è un virus appartenente alla famiglia degli Herpesviridae. Esistono due tipi principali di HSV: l'HSV-1 e l'HSV-2. L'HSV-1 è comunemente associato all'herpes labiale, mentre l'HSV-2 è responsabile dell'herpes genitale. Entrambi i tipi di HSV possono causare infezioni ricorrenti e dolorose, caratterizzate da vesciche che si formano sulla pelle o sulle mucose. Non esiste una cura definitiva per l'HSV, ma possono essere utilizzati farmaci antivirali per ridurre la durata e la gravità delle infezioni.


## SimpleSequentialChains

Permettono di fare una serie di calls a uno o più LLMs, l'ouput di una chain è l'input di un'altra.
Nelle **Simple** ogni singola catena ha un solo input e output, che costituisce l'input della catena sucessiva.


In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=1024)
prompt1 = PromptTemplate(
    input_variables=['concept'],
    template='''You are an experienced scientist and Python programmer.
    Write a function that implements the concept of {concept}.'''
)
chain1 = LLMChain(llm=llm1, prompt=prompt1)


llm2 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1.2)
prompt2 = PromptTemplate(
    input_variables=['function'],
    template='Given the Python function {function}, describe it as detailed as possible.'
)
chain2 = LLMChain(llm=llm2, prompt=prompt2)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
output = overall_chain.run('linear regression') #Si noti che viene passato solo l'argomento della prima catena



> Entering new SimpleSequentialChain chain...


def linear_regression(x, y):
    """
    Calculates linear regression of two sets of data points
    
    Parameters:
    x: A list of x values
    y: A list of y values
    
    Returns:
    A tuple (m, b) of the slope and intercept of the line of best fit
    """
    # Calculate the means
    x_mean = sum(x) / len(x)
    y_mean = sum(y) / len(y)

    # Calculate the numerator and denominator of the slope
    numerator = 0
    denominator = 0
    for i in range(len(x)):
        numerator += (x[i] - x_mean) * (y[i] - y_mean)
        denominator += (x[i] - x_mean) ** 2
    
    # Calculate the slope and intercept
    m = numerator / denominator
    b = y_mean - m * x_mean
    
    return (m, b)
The given Python function named "linear_regression" accepts two lists named "x" and "y" as input parameters. These lists represent the x-values and y-values of a set of data points, respectively. 

The function calculates the linear regression of 

### LangChain Agents

In [23]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool  
from langchain.llms import OpenAI

In [24]:
llm = OpenAI(temperature=0)
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

agent_executor.run('what is the answer to 5.1 ** 7.3?')



> Entering new AgentExecutor chain...
 I need to use the Python REPL to calculate this
Action: Python_REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought: I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


'146306.05007233328'

In [25]:
agent_executor.run('Calculate the square root of the factorial of 20 and display it with 4 decimal points')



> Entering new AgentExecutor chain...
 I need to calculate the factorial of 20 and then take the square root of that
Action: Python_REPL
Action Input: from math import factorial; print(round(factorial(20)**0.5, 4))
Observation: 1559776268.6285

Thought: I now know the final answer
Final Answer: 1559776268.6285

> Finished chain.


'1559776268.6285'